# Introduction to TMS and Motor Thresholds 

Authored by Remy Cohan [cohanR.github.io] with minor edits - mostly formatting - by Peter Kohler [kohlerlab.com].

As discussed during the lecture and as you saw in the video we played during class (also available on eClass), in TMS research, we often measure motor thresholds (MTs) using single-pulse TMS to primary motor cortex (M1). The resting motor threshold (rMT) is defined as the minimum TMS intensity that produces an electromyographic (EMG) response of 50 µV in a relaxed muscle. MTs are an important measure because they can reflect changes in neural excitability and plasticity.

On the other hand, repetitive TMS (rTMS) is a different technique where multiple pulses are delivered in patterns (e.g., 1 Hz, 10 Hz, or theta burst stimulation). rTMS is gaining popularity as a potential treatment for neuropsychiatric conditions and in neurorehabilitation.

To study the effects of rTMS, we can measure MTs using single-pulse TMS before and after an rTMS protocol. This allows us to see how the rTMS protocol changes neural activity in a specific brain area, which is critical when designing treatment protocols.

Remember the distinction:

* Single-pulse TMS: Measures MTs and neural excitability in primary motor cortex (M1).
    
* Repetitive TMS (rTMS): Modulates neural activity to induce plasticity.
    
In the data below, pre-rTMS refers to a baseline single-pulse TMS measure of MTs before the rTMS protocol. Post-rTMS refers to the same measure after the rTMS protocol.

This background should help you interpret the data and answer the following questions.

**It is a good idea to review some of Panda's functions such as groupby and .agg here:** https://github.com/marsja/jupyter/blob/master/pandas_groupby_tutorial.ipynb

## Preample: Import some packages 

As in often the case, we start our code by importing some Python modules. 

In this case, we import the pandas module, which is a powerful data manipulation library and scipy which we use for our correlation analysis. 
We also import modules for plotting, matplotlib and seaborn. 

In [39]:
# install tools for getting data off osf, and for fitting the data
try:
  import google.colab
  IN_COLAB = True
except:
  IN_COLAB = False
if IN_COLAB:  
    import shutil
    # download github repository to get access to the tms data
    shutil.rmtree("NRSC2200", ignore_errors=True)
    !git clone https://github.com/pjkohler/NRSC2200
    # add tms directory to path
    datapath = "NRSC2200/tms/tms_cohan_data.csv"
else:
    datapath = "tms_cohan_data.csv"
# import python modules to use for analysis
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy.stats import pearsonr

## Question 1: Data Inspection (1 point)

How many pre- and post data points are present in the dataset?

To answer this, load the dataset and count the number of available data points for the pre_rMT and post_rMT columns.

Remember, pandas was imported as pd (see above). The code below starts with reading the data from the csv file into a pandas dataframe. 

Please include only the lines of code that you changed, in your submitted answer.

In [ ]:
# load the dataset
data = pd.read_csv(datapath)

# print column names from the data (csv file):

# Where ever you see this "# add your code here" you need to add your answer 

# in this first example, we are going to give you the answer:

print("column names:", # add your code here )    # answer: data.columns.tolist()

In [ ]:
# Using the variable names pre_count and post_count, count the number of pre- and post-rMT data points and print the answer
# Hint: Use .notna() to check for missing values before counting.

# fill in the function to count non-null values for pre_rMT
pre_count = # add your code here

# fill in the function to count non-null values for post_rMT      
post_count =  # add your code here

# if the code you added above are correct, running this section should print out the answers
print(f"number of Pre rMT data points: {pre_count}")
print(f"number of Post rMT data points: {post_count}")

## Question 2: Check for missing values (1 point)

Please include only the lines of code that you changed, in your submitted answer.

In [ ]:
# fill in the function to check for missing values in the data
# hint: The .isnull().sum() method shows missing values for each column.

missing_values =  # add your code here 

# if the code you added above is correct, running this section should print out the answers
print("missing values in each column:")
print(missing_values)

## Question 3: Participant Demographics (2 point)

What are the demographic statistics of the participants? 

Specifically: How many males and females are there in the dataset?

What are the mean and standard deviation of age, pre-rMT, and post-rMT values?

In [ ]:
# add code to count males and females
# hint: The .value_counts() function helps count categorical variables.

sex_counts = # add your code here  

# if the code you added above is correct, running this section should print out the answers
print("count of participants by sex:")
print(sex_counts)

# compute mean and standard deviation for pre- and post-rMT values by condition
# hint: Use .groupby() to separate data by condition before applying statistics.

stats_summary = # add your code here  

# if the function you added above is correct, running this section should print out the answers
print("mean and standard deviation for Pre MTs, and Post MTs by condition:")
print(stats_summary)

## Question 4: Pre vs post rMT values by Condition (1 point)

How do the pre- and post-rMT values compare between the active and sham conditions?

Create a plot to visualise the differences between the two conditions. ( use seaborn or matplotlib, or both. e'g., import matplotlib.pyplot as plt
import seaborn as sns)

In [ ]:
# create boxplot comparing pre- and post-rMT values by condition
# hint: Convert data into long format using pd.melt() before plotting.

# in the code snippet below, where ever you see 3 hastags (###) add the correct column name for the correct plotting of the data

tms_data = pd.melt(data, id_vars =['participant_id', ###],  
                      value_vars=[###, ###],            
                      var_name='time', value_name='rMT')
plt.figure(figsize=(10, 6))
sns.boxplot(x='condition', y='rMT', hue='time', data=tms_data, palette='Set2')
plt.title("Pre vs Post rMT by Condition")
plt.xlabel("Condition")
plt.ylabel("rMT (TMS dose)")
plt.legend(title="Time")
plt.show()

## Question 5: Correlation between Age and baseline rMT (2 points)

Run the code below to plot age against baseline rMT - no need to make any changes to the code. 

Look at the plot and decide whether you think it is likely that there is a significant correlation between participants' age and their pre-rMT values.

Please report what you concluded, and briefly explain why. 

In [ ]:
# correlation plot for age vs pre rMTs
corr, pval = pearsonr(data['age'], data['pre_rMT'])

plt.figure(figsize=(8, 6))
sns.regplot(x='age', y='pre_rMT', data=data, scatter_kws={'s': 50, 'alpha': 0.7, 'clip_on': False}, color='green')
plt.title("age vs baseline rMT ")
plt.xlabel("age (years)")
plt.ylabel("baseline rMT")
plt.xlim(20, 60)
plt.ylim(45, 75)
plt.text(35, 50, f"correlation between age and pre rMT:\nr = {corr:.2f}", fontsize=12)
plt.show()

## Question 6: Interpretation (2 points)

Consider the mean pre- to post-rMT values for Active TMS vs Sham TMS that you plotted in Question 4.

Without performing any statistics, just by inspecting the plots and mean values, what conclusion would you draw?

What does the correlation plot of age and pre-rMT values tell us?

When answering these questions, remember that pre-rTMS refers to a baseline single-pulse TMS measure of MTs before the rTMS protocol, while post-rTMS refers to the same measure after the rTMS protocol.
